<a href="https://colab.research.google.com/github/kashyap333/Tabular-Playground-Series---Aug-2022/blob/main/Tabular_Playground_Series_Aug_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle setup and data download

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c tabular-playground-series-aug-2022


In [ ]:
!unzip tabular-playground-series-aug-2022

In [ ]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

# Data Loading and visualising with pandas profiling


In [72]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
from sklearn.ensemble import VotingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer, LabelEncoder
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport as pr
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression, PoissonRegressor, SGDClassifier
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import make_pipeline, Pipeline
from pandas_profiling import ProfileReport
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter 
%matplotlib inline

In [7]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train_test = pd.concat([train[test.columns], test], ignore_index=True)

In [ ]:
pr = ProfileReport
report = pr(train_test, title='profile result', html={'style': {'full_width': True}})
report.to_notebook_iframe()

In [ ]:
train_test.head()

In [ ]:
train_test.info()

# Data Preprocessing

In [8]:
object_columns = ['product_code', 'attribute_0', 'attribute_1']
train_test.drop(['id'], axis=1, inplace=True)
#train_test_object = train_test.copy()
train_test.drop(object_columns, axis=1, inplace=True)
all_columns = train_test.columns

lb = LabelEncoder()
for i in train_test_object.columns:
  train_test_object[i] = lb.fit_transform(train_test_object[i])

In [9]:
imp = IterativeImputer(max_iter=10, initial_strategy='mean')
std_scaler = StandardScaler()
for i in train_test.columns:
  
  train_test[[i]] = imp.fit_transform(train_test[[i]])

train_test[all_columns] = std_scaler.fit_transform(train_test[all_columns])

In [174]:
#full_data = pd.concat([train_test_object, train_test], axis=1)

In [10]:
full_data = train_test.copy()

In [11]:
x_train = full_data.iloc[:train.shape[0], :]
y_train = train.failure.astype('float')
x_test = full_data.iloc[train.shape[0]:, :]

In [13]:
sm = SMOTE(random_state=42)
x_train, y_train = sm.fit_resample(x_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train))

Resampled dataset shape Counter({0.0: 20921, 1.0: 20921})


# Model Building


In [77]:
ensemble = VotingClassifier(
    estimators=[
        ('lr',  LogisticRegression(class_weight='balanced',random_state = 0)),
        ("modelrfc", RandomForestClassifier(
        class_weight='balanced', 
        criterion='entropy',
        max_depth=22, 
        min_samples_split=4, 
        n_estimators=250,
        random_state=56)),
        ('gaussian', GaussianNB()),
        ('KNN', KNeighborsClassifier()),
        ('SVM', SVC(probability=True)),
        ('xgb', XGBClassifier())
    ], voting='soft')

steps = [('ec',ensemble)]
         

In [78]:
stacking_classifiers = Pipeline(steps)

In [79]:
stacking_classifiers= stacking_classifiers.fit(x_train, y_train)

In [80]:
trainy_pred_ec = stacking_classifiers.predict(x_train)
validy_pred_ec = stacking_classifiers.predict_proba(x_test)

In [81]:
score = metrics.roc_auc_score(y_train, trainy_pred_ec)
print(f'The error for roc_auc  : {score}')

The error for roc_auc  : 0.9040915826203337


In [ ]:
sklearn.metrics.SCORERS.keys()

In [ ]:
'''
loss = []
for name, model in models.items():
  cv_results = cross_validate(model, X=x_train, y=y_train, cv=5, scoring=scoring)
  clf = model.fit(x_train, y_train)
  pred = model.predict(x_train)
  score = metrics.roc_auc_score(y_train, pred)
  print(f'The error for {scoring} with {model} : {score}')
'''

In [82]:
pred = clf.predict_proba(x_test)

# Submission to Kaggle

In [83]:
submission = pd.DataFrame(test['id'])
submission['failure'] = pred[:, 1]

In [84]:
submission.to_csv('submission2.csv', index=False)

In [ ]:
!kaggle competitions submit -c tabular-playground-series-aug-2022 -f submission.csv -m "5"

In [ ]:
submission.head()